### 🔓 LLL Attack

Assume the message has the form m = B + x, where x is a small unknown.

Steps:
- Construct the polynomial: \\( f(T) = (B + T)^e - c \; mod n \\)
- Build a lattice using scaled basis vectors
- Apply LLL to find a short vector interpreted as \\( g(T) \\)
- Solve \\( g(x) = 0 \\) over the integers
- Recover \\( m = B + x \\) and verify \\( pow(m, e, n) == c \\)

#### 📚 Reference:
Trappe, W. and L. C. Washington (2006). *Introduction to Cryptography with Coding Theory*, 2nd ed.
**Chapter 17.3 – An Attack on RSA**, Upper Saddle River: Pearson.



In [2]:
from sympy import symbols, Poly, solve, im
from lattice_methods import lll_reduce
import numpy as np

n = 1927841055428697487157594258917
e = 3
Y = 100 #   |x| <= Y (xmax)
B = 200805000114192305180009190000
c = 30326308498619648559464058932

print("🔐 RSA Setup:")
print(f"   • Public modulus (n): {n}")
print(f"   • Public exponent (e): {e}")
print(f"   • Ciphertext (c): {c}")
print()
print("📦 Message structure:")
print(f"   • Assumed form: m = B + x")
print(f"   • Known prefix (B): {B}")
print(f"   • Unknown root (x): ?")
print(f"   • Bound on x: |x| <= {Y}")


a0 = pow(B,3,n) - c
a1 = (3 * pow(B,2,n)) % n
a2 = (3 * B) % n
a3 = 1

#print(a0, a1, a2, a3)


v1 = np.array([n, 0, 0, 0])
v2 = np.array([0, n * Y, 0, 0])
v3 = np.array([0, 0, n * pow(Y,2,n), 0])
v4 = np.array([a0, a1*Y, a2*pow(Y,2,n), a3*pow(Y,3,n)])


basis = [v1, v2, v3, v4]

reduced = lll_reduce(basis)
coeffs = reduced[0]


# print(v1)
# print(v2)
# print(v3)
# print(v4)

e0 = int(coeffs[0])
e1 = int(coeffs[1]) // Y
e2 = int(coeffs[2]) // (Y**2)
e3 = int(coeffs[3]) // (Y**3)

T = symbols('T')
g = Poly(e3*T**3 + e2*T**2 + e1*T + e0)
roots = solve(g)

c_check = 0
root = 0

for r in roots:
    if(im(r) != 0 or r >= B or r < 0):
        continue
    else:
        root = r
        c_check = pow((B+r),e,n)

print()
print("🧾 Final Result Summary")
print("────────────────────────────────────────────")

print(f"📍 Recovered root x: {root}")
print(f"📍 Reconstructed message m = B + x = {B} + {root} = {B + root}\n")
print(f"🧮 Computed ciphertext: c' = pow(m, e, n) = {c_check}")
print(f"🎯 Target ciphertext:    c  = {c}")
print()

if c_check == c:
    print("✅ SUCCESS: LLL attack successfully recovered the correct message!")
else:
    print("❌ FAILURE: Recovered message does not match the original ciphertext.")

#TODO might have to add this for additional check up
#print(g(42))

🔐 RSA Setup:
   • Public modulus (n): 1927841055428697487157594258917
   • Public exponent (e): 3
   • Ciphertext (c): 30326308498619648559464058932

📦 Message structure:
   • Assumed form: m = B + x
   • Known prefix (B): 200805000114192305180009190000
   • Unknown root (x): ?
   • Bound on x: |x| <= 100

🧾 Final Result Summary
────────────────────────────────────────────
📍 Recovered root x: 42
📍 Reconstructed message m = B + x = 200805000114192305180009190000 + 42 = 200805000114192305180009190042

🧮 Computed ciphertext: c' = pow(m, e, n) = 30326308498619648559464058932
🎯 Target ciphertext:    c  = 30326308498619648559464058932

✅ SUCCESS: LLL attack successfully recovered the correct message!
